In [ ]:
# # **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI & SKTIME ****************
stable = False # Set to True for latest pip version or False for main branch in GitHub
!pip install tsai
!pip install sktime -U  >> /dev/null

In [ ]:
!pip install ipywidgets

In [3]:
from tsai.basics import *
import sktime
import sklearn
import numpy as np
my_setup(sktime, sklearn)

os             : Linux-5.13.0-41-generic-x86_64-with-glibc2.17
python         : 3.8.13
tsai           : 0.3.1
fastai         : 2.6.3
fastcore       : 1.4.3
sktime         : 0.11.4
sklearn        : 1.1.1
torch          : 1.11.0+cu102
device         : cpu
cpu cores      : 16
RAM            : 15.0 GB
GPU memory     : N/A


In [2]:
from tsai.models.MINIROCKET import *

In [3]:
path_to_data = "./"

In [4]:
ls

actual_data_predict.ipynb         prediction_for_10_days.ipynb
data/                             preprocessing/
data.json                         snow.ipynb
dockerfile                        tree_reggression.ipynb
minirocket_tests_with_snow.ipynb  visualisation.py
pavodki_inference.ipynb


In [5]:
df = pd.read_csv('data/data_2022/only_zeya_fixed_actual_data.csv')

In [6]:
# df = df.drop(['snow_high', 'snow_level', 'snow_Q1'], axis=1)
df

,Unnamed: 0,zeya-malayasazanka,zeya-svobodny,zeya-blagoveshensk,zeya-mazanovo,zeya-belogorye
0,2008-01-01,435.0,163.0,148.0,218.0,341.0
1,2008-01-02,441.0,162.0,151.0,222.0,350.0
2,2008-01-03,447.0,161.0,157.0,226.0,361.0
3,2008-01-04,447.0,154.0,163.0,230.0,367.0
4,2008-01-05,443.0,144.0,168.0,222.0,364.0
...,...,...,...,...,...,...
5254,2022-05-21,500.0,227.0,169.0,219.0,393.0
5255,2022-05-22,533.0,247.0,174.0,203.0,412.0
5256,2022-05-23,533.0,234.0,181.0,228.0,412.0
5257,2022-05-24,522.0,224.0,190.0,324.0,404.0


In [7]:
X = df[df.keys()[1:]].to_numpy()
X

array([[435., 163., 148., 218., 341.],
       [441., 162., 151., 222., 350.],
       [447., 161., 157., 226., 361.],
       ...,
       [533., 234., 181., 228., 412.],
       [522., 224., 190., 324., 404.],
       [570., 298., 196., 372., 446.]])

In [8]:
X.shape

(5259, 5)

In [9]:
number_of_frames = X.shape[1]


In [10]:
def create_dataset(df, num_days=365):
    number_of_frames = df[df.keys()[2:]].to_numpy().shape[1]
    X = df[df.keys()[1:]].to_numpy().reshape(-1, number_of_frames, 1)
    array_for_concat = []
    for i in range(num_days):
        array_for_concat.append(X[i:X.shape[0]-(num_days - i)])
    new_X = np.concatenate(array_for_concat, axis=-1)
    y = df['zeya-blagoveshensk'].to_numpy()[num_days:]
    
    return new_X, y

In [11]:
def create_multiple_dataset(df, num_days=365, days_forward=1):
    number_of_frames = df[df.keys()[1:]].to_numpy().shape[1]
    X = df[df.keys()[1:]].to_numpy().reshape(-1, number_of_frames, 1)
    array_for_concat = []
    for i in range(num_days):
        array_for_concat.append(X[i:X.shape[0]-(num_days - i)-days_forward+1])
    new_X = np.concatenate(array_for_concat, axis=-1)
    y = df['zeya-blagoveshensk'].to_numpy().reshape(-1, 1)
    array_for_concat = []
    for i in range(days_forward):
        array_for_concat.append(y[num_days+i:y.shape[0] - (days_forward - i - 1)])
    new_y = np.concatenate(array_for_concat, axis=-1)
    
    return new_X, new_y

In [12]:
def create_multiple_dataset_preprocessed(df, num_days=365, days_forward=1):
    number_of_frames = df[df.keys()[2:]].to_numpy().shape[1]
    X = df[df.keys()[1:]].to_numpy().reshape(-1, number_of_frames, 1)
    array_for_concat = []
    for i in range(num_days):
        array_for_concat.append(X[i:X.shape[0]-(num_days - i)-days_forward+1])
    new_X = np.concatenate(array_for_concat, axis=-1)
    y = df['zeya-blagoveshensk'].to_numpy().reshape(-1, 1)
    array_for_concat = []
    for i in range(days_forward):
        array_for_concat.append(y[num_days+i:y.shape[0] - (days_forward - i - 1)])
    new_y = np.concatenate(array_for_concat, axis=-1)
    
    return new_X / np.linalg.norm(new_X), new_y / np.linalg.norm(new_y)

In [13]:
def create_single_dataset(df, num_days=365, days_forward=1):
    number_of_frames = df[df.keys()[2:]].to_numpy().shape[1]
    X = df[df.keys()[1:]].to_numpy().reshape(-1, number_of_frames, 1)
    array_for_concat = []
    for i in range(num_days):
        array_for_concat.append(X[i:X.shape[0]-(num_days - i)-days_forward+1])
    new_X = np.concatenate(array_for_concat, axis=-1)
    y = df['zeya-blagoveshensk'].to_numpy().reshape(-1, 1)
    array_for_concat = []
    for i in range(days_forward):
        array_for_concat.append(y[num_days+i:y.shape[0] - (days_forward - i - 1)])
    new_y = np.concatenate(array_for_concat, axis=-1)
    return new_X, new_y[:, -1]

In [14]:
def create_multiple_dataset_preprocessed(df, num_days=365, days_forward=1):
    number_of_frames = df[df.keys()[2:]].to_numpy().shape[1]
    X = df[df.keys()[1:]].to_numpy().reshape(-1, number_of_frames, 1)
    array_for_concat = []
    for i in range(num_days):
        array_for_concat.append(X[i:X.shape[0]-(num_days - i)-days_forward+1])
    new_X = np.concatenate(array_for_concat, axis=-1)
    y = df['zeya-blagoveshensk'].to_numpy().reshape(-1, 1)
    array_for_concat = []
    for i in range(days_forward):
        array_for_concat.append(y[num_days+i:y.shape[0] - (days_forward - i - 1)])
    new_y = np.concatenate(array_for_concat, axis=-1)
    
    return new_X / np.linalg.norm(new_X), new_y / np.linalg.norm(new_y)

In [15]:
X, y = create_multiple_dataset(df, 365, 1)

In [16]:
X.shape, y.shape

((4894, 5, 365), (4894, 1))

In [17]:
y

array([[143.],
       [148.],
       [140.],
       ...,
       [181.],
       [190.],
       [196.]])

In [18]:
X = df1['Value'].to_numpy()
X = X.reshape(-1, 1)

NameError: name 'df1' is not defined

In [19]:
def make_ts(n):
    res = []
    for i in range(n + 1):
        res.append(X[i:6401-(n - i)])
    return res

In [1]:
def make_dataset_from_x(n):
    X11 = np.concatenate(np.array(make_ts(n)), axis=1)[:-1].reshape((-1, 1, n + 1))
    y1 = X[n + 1:]
    return X11, y1

In [2]:
X1, y = make_dataset_from_x(364)

NameError: name 'np' is not defined

In [ ]:
1 - float((X.shape[0] - 10) / X.shape[0])

In [ ]:
from sklearn.model_selection import train_test_split
valid_leng = 1 - float((X.shape[0] - 10) / X.shape[0])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, shuffle=True)
X_train, X_, y_train, y_ = train_test_split(X_train, y_train, test_size=1, shuffle=True)
#X_train, X_valid, y_trai, y_valid = train_test_split(X, y, , shuffle=False)
y_.shape

In [ ]:
# Univariate regression with sklearn-type API
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
model = MiniRocketRegressor(scoring=rmse_scorer)
print(model)
timer.start(False)
model.fit(X_train, y_train)
t = timer.stop()
y_pred = model.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
mae = mean_absolute_error(y_valid, y_pred)
print(f'valid rmse        : {rmse:.5f} time: {t}')
print(f'valid mae        : {mae:.5f} time: {t}')

In [ ]:
#make grahs
import matplotlib.pyplot as plt
valid_value = []
predicted_value = []
for i in range(len(y_valid[0])):
    valid_value.append([])
    predicted_value.append([])
for valid, pred in zip(y_valid, y_pred):
    for i in range(len(valid)):
        valid_value[i].append(valid[i])
        predicted_value[i].append(pred[i])
for valid, pred in zip(valid_value, predicted_value):
    plt.plot(valid, color='r', label='valid')
    plt.plot(pred, color='g', label='predicted')
    plt.legend()
    plt.show()

In [ ]:
#make grahs for extreeme levels
import matplotlib.pyplot as plt
CRITICAL_LEVEL = 510
valid_value = []
predicted_value = []
for i in range(len(y_valid[0])):
    valid_value.append([])
    predicted_value.append([])
for valid, pred in zip(y_valid, y_pred):
    for i in range(len(valid)):
        if valid[i] >= CRITICAL_LEVEL:
            valid_value[i].append(valid[i])
            predicted_value[i].append(pred[i])
for valid, pred in zip(valid_value, predicted_value):
    plt.plot(valid, color='r', label='valid')
    plt.plot(pred, color='g', label='predicted')
    plt.legend()
    plt.show()

In [ ]:
error_value = []
for i in range(len(y_valid[0])):
    error_value.append( mean_absolute_error(y_valid[:, i], y_pred[:, i]) / y_valid[:, i])
plt.plot(error_value, color='r', label='error_change')
plt.legend()
plt.show()

In [ ]:
np.mean(mean_absolute_error(y_valid[:, i], y_pred[:, i]) / y_valid[:, i])

In [ ]:
mean_absolute_error(y_valid[:][0], y_pred[:][0])

In [ ]:
# Multivariate regression ensemble with sklearn-type API
from sklearn.metrics import mean_squared_error, make_scorer
dsid = 'AppliancesEnergy'
#X_train, y_train, X_valid, y_valid = get_Monash_regression_data(dsid)
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
model = MiniRocketVotingRegressor(n_estimators=5, scoring=rmse_scorer)
timer.start(False)
model.fit(X_train, y_train)
t = timer.stop()
y_pred = model.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
print(f'valid rmse        : {rmse:.5f} time: {t}')

Git Gud

In [ ]:
X1, y = make_dataset_from_x(364)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X1, y, test_size=0.3)
# Univariate regression with sklearn-type API
from sklearn.metrics import mean_squared_error, make_scorer
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
model = MiniRocketRegressor(scoring=rmse_scorer)
timer.start(False)
model.fit(X_train, y_train)
t = timer.stop()
y_pred = model.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
print(f'valid rmse        : {rmse:.5f} time: {t}')

In [ ]:
X1, y = make_dataset_from_x(364)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X1, y, test_size=0.3)
# Univariate regression with sklearn-type API
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
rmse_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
model = MiniRocketRegressor(scoring=rmse_scorer)
timer.start(False)
model.fit(X_train, y_train)
t = timer.stop()
y_pred = model.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
rmae = mean_absolute_error(y_valid, y_pred)
print(f'valid rmae        : {rmae:.5f} time: {t}')
print(f'valid rmse        : {rmse:.5f} time: {t}')

In [ ]:
X1, y = make_dataset_from_x(364)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X1, y, test_size=0.3)
# Univariate regression with sklearn-type API
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
model = MiniRocketRegressor(scoring=rmse_scorer)
timer.start(False)
model.fit(X_train, y_train)
t = timer.stop()
y_pred = model.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
rmae = mean_absolute_error(y_valid, y_pred)
print(f'valid rmae        : {rmae:.5f} time: {t}')
print(f'valid rmse        : {rmse:.5f} time: {t}')

In [ ]:
# Multivariate regression ensemble with sklearn-type API
from sklearn.metrics import mean_squared_error, make_scorer
dsid = 'AppliancesEnergy'
#X_train, y_train, X_valid, y_valid = get_Monash_regression_data(dsid)
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
model = MiniRocketVotingRegressor(n_estimators=5, scoring=rmse_scorer)
timer.start(False)
model.fit(X_train, y_train)
t = timer.stop()
y_pred = model.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
rmae = mean_absolute_error(y_valid, y_pred)
print(f'valid rmae        : {rmae:.5f} time: {t}')
print(f'valid rmse        : {rmse:.5f} time: {t}')

In [ ]:
# Multivariate regression ensemble with sklearn-type API
from sklearn.metrics import mean_squared_error, make_scorer
dsid = 'AppliancesEnergy'
#X_train, y_train, X_valid, y_valid = get_Monash_regression_data(dsid)
rmse_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
model = MiniRocketVotingRegressor(n_estimators=5, scoring=rmse_scorer)
timer.start(False)
model.fit(X_train, y_train)
t = timer.stop()
y_pred = model.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
rmae = mean_absolute_error(y_valid, y_pred)
print(f'valid rmae        : {rmae:.5f} time: {t}')
print(f'valid rmse        : {rmse:.5f} time: {t}')

In [ ]:
!pip install pycaret
